In [15]:
import numpy as np

os.chdir("/home/david/Documents/ghsom")

filename = "HI-II-14.tsv"

lines = []

for line in open(filename):
    lines.append(line.rstrip().split("\t"))

with open("HI-II-14.txt", "w") as f:
    first = True
    for line in lines:
        if first:
            first=False
            continue
        f.write("{} {}\n".format(line[0], line[2]))

In [1]:
import networkx as nx
G = nx.read_edgelist("Uetz_screen.txt")

print len(G.nodes())

806


In [1]:
from save_embedded_graph27 import main as embed_main

embed_main('HI-II-14.txt', 'embedded_hi_ii_14.gml')

In [1]:
from spearmint_ghsom import main_no_labels as ghsom_main
import pickle
import os

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

    
os.chdir("/home/david/Documents/ghsom")

init = 1
p = 0.7

#ghsom parameters
params = {'w': 0.0001,
         'eta': 0.001,
         'sigma': 1,
          'e_sg': p,
         'e_en': 10}



In [ ]:
# G, map = ghsom_main(params, 'embedded_hi_ii_14.gml')
G, map = ghsom_main(params, 'embedded_yeast_union.gml', init=init, lam=1000)
# G, map = ghsom_main(params, 'embedded_yeast_uetz.gml', init=init, lam=1000)

print '\nnumber of communities detected: {}'.format(len(map))
# save_obj((G, map), 'HI_II_communities_{}'.format(p))
# save_obj((G, map), 'yeast_union_communities_{}'.format(p))
save_obj((G, map), 'yeast_uetz_communities_{}_{}'.format(p, init))

print 'done'

In [5]:
import os

os.chdir("/home/david/Documents/ghsom")

# G, map = load_obj('HI_II_communities_{}'.format(p))
# G, map = load_obj('yeast_union_communities_{}'.format(p))
G, map = load_obj('yeast_union_communities_{}'.format(p))
print 'number of communities detected: {}'.format(len(map))

number of communities detected: 20


In [7]:
import networkx as nx

for n, d in map.nodes(data=True):
    
    H = G.subgraph(d['ls'])
    print len(H)
    print nx.is_connected(H)

34
False
58
False
146
False
88
True
146
False
72
True
188
True
102
False
194
True
27
False
65
False
75
True
103
True
51
True
50
True
67
True
43
True
37
True
52
True
49
True


In [12]:
min_nodes = 10

##remove neurons with no assigned nodes
for n, d in map.nodes(data=True):
    if len(d['ls']) < min_nodes:
        map.remove_node(n)
        print 'removed node {}'.format(n)

In [5]:
import os
import networkx as nx
import numpy as np
##save to communities directory
os.chdir("/home/david/Documents/ghsom")

# dir_name = "union_communities_08"
dir_name = "uetz_communities_{}_{}".format(p, init)

if not os.path.isdir(dir_name):
    os.mkdir(dir_name)
    print 'made directory {}'.format(dir_name)
    
os.chdir(dir_name)

shortest_path = nx.floyd_warshall_numpy(map).astype(np.int)
np.savetxt("shortest_path.csv", shortest_path, fmt='%i', delimiter=",")
print 'written shortest path matrix'

c = 0
for n, d in map.nodes(data=True):
    ls = d['ls']
    with open('community_{}.txt'.format(c),'w') as f:
        for l in ls:
            f.write('{}\n'.format(l))
    print 'written community_{}.txt'.format(c)
    c += 1

written shortest path matrix
written community_0.txt
written community_1.txt
written community_2.txt
written community_3.txt
written community_4.txt
written community_5.txt
written community_6.txt
written community_7.txt
written community_8.txt
written community_9.txt
written community_10.txt
written community_11.txt
written community_12.txt
written community_13.txt
written community_14.txt
written community_15.txt
written community_16.txt
written community_17.txt
written community_18.txt
written community_19.txt
written community_20.txt
written community_21.txt
written community_22.txt
written community_23.txt
written community_24.txt


In [11]:
import os
import networkx as nx
import numpy as np
from spearmint_ghsom import main_no_labels as ghsom_main
import pickle
import shutil

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
root_dir = "/home/david/Documents/ghsom"

data = "yeast_uetz_rel"
init = 1

for p in np.arange(0.1, 1, 0.1)[::-1]:
    
    print "p={}".format(p)
    
    os.chdir(root_dir)
    
    #ghsom parameters
    params = {'w': 0.0001,
             'eta': 0.001,
             'sigma': 1,
              'e_sg': p,
             'e_en': 10}
    
    map_file = '{}_communities_{}_{}'.format(data, p, init)
    
    if not os.path.isfile("{}.pkl".format(map_file)):
    
        #run ghsom and save output
        print "running GHSOM and saving to {}.pkl".format(map_file)
        G, map = ghsom_main(params, 'embedded_{}.gml'.format(data), init=init, lam=1000)
        print '\nnumber of communities detected: {}, saved map to {}'.format(len(map), map_file)
        save_obj((G, map), map_file)
    
    else:
        
        print "{}.pkl already exists, loading map".format(map_file)    
        #load output
        G, map = load_obj(map_file)

    #save results to file
    dir_name = "{}_communities_{}_{}".format(data, p, init)
    if not os.path.isdir(dir_name):
#         shutil.rmtree(dir_name)
#         print "deleted directory {}".format(dir_name)
    
        os.mkdir(dir_name)
        print 'made directory {}'.format(dir_name)

    os.chdir(dir_name)
    print "moved to {}".format(dir_name)
    
    #all genes
    all_genes_file = "all_genes.txt"
    with open(all_genes_file, 'w') as f:
        for n in G.nodes():
            f.write("{}\n".format(n))
    print "written {}".format(all_genes_file)
    
    #save shortest path matrix
    shortest_path = nx.floyd_warshall_numpy(map).astype(np.int)
    np.savetxt("shortest_path.csv", shortest_path, fmt='%i', delimiter=",")
    print 'written shortest path matrix'
    
    #save communities to file
    c = 0
    for n, d in map.nodes(data=True):
        ls = d['ls']
        with open('community_{}.txt'.format(c),'w') as f:
            for l in ls:
                f.write('{}\n'.format(l))
        print 'written community_{}.txt'.format(c)
        c += 1
    print

p=0.9
running GHSOM and saving to yeast_uetz_rel_communities_0.9_1.pkl
Layer: 1, training epoch: 999/1000, size of map: 2, MQE: 0.693286655558, target: 0.623630575367, sigma: 0.135606224654          
number of communities detected: 2, saved map to yeast_uetz_rel_communities_0.9_1
made directory yeast_uetz_rel_communities_0.9_1
moved to yeast_uetz_rel_communities_0.9_1
written all_genes.txt
written shortest path matrix
written community_0.txt
written community_1.txt

p=0.8
running GHSOM and saving to yeast_uetz_rel_communities_0.8_1.pkl
Layer: 1, training epoch: 999/1000, size of map: 3, MQE: 0.578922321711, target: 0.556141650101, sigma: 0.135606224654          
number of communities detected: 3, saved map to yeast_uetz_rel_communities_0.8_1
made directory yeast_uetz_rel_communities_0.8_1
moved to yeast_uetz_rel_communities_0.8_1
written all_genes.txt
written shortest path matrix
written community_0.txt
written community_1.txt
written community_2.txt

p=0.7
running GHSOM and saving to 

KeyboardInterrupt: 